# Kafka Consumer

In [13]:
import requests
import json
import time

# Kafka REST Proxy URL 설정
KAFKA_REST_PROXY_URL = "http://localhost:8082"
CONSUMER_GROUP = "my_consumer_group"
TOPIC = "my_topic"

# Consumer 인스턴스 생성
consumer_data = {
    "name": "my_consumer_instance",
    "format": "json",
    "auto.offset.reset": "earliest",
}

response = requests.post(
    f"{KAFKA_REST_PROXY_URL}/consumers/{CONSUMER_GROUP}",
    headers={"Content-Type": "application/vnd.kafka.v2+json"},
    data=json.dumps(consumer_data),
)
response.json()

{'instance_id': 'my_consumer_instance',
 'base_uri': 'http://localhost:8082/consumers/my_consumer_group/instances/my_consumer_instance'}

In [14]:
if response.status_code == 200:
    consumer_instance = response.json()
    print("Consumer instance created successfully")
else:
    print(
        f"Failed to create consumer instance: {response.status_code}, {response.text}"
    )
    exit(1)

# 토픽 구독
subscription_data = {"topics": [TOPIC]}

response = requests.post(
    f"{consumer_instance['base_uri']}/subscription",
    headers={"Content-Type": "application/vnd.kafka.v2+json"},
    data=json.dumps(subscription_data),
)

if response.status_code == 204:
    print("Subscribed to topic successfully")
else:
    print(f"Failed to subscribe to topic: {response.status_code}, {response.text}")
    exit(1)

# 메시지 읽기
try:
    while True:
        response = requests.get(
            f"{consumer_instance['base_uri']}/records",
            headers={"Accept": "application/vnd.kafka.json.v2+json"},
            timeout=5,
        )

        if response.status_code == 200:
            records = response.json()
            for record in records:
                print(f"Received message: {record['value']}")
        elif response.status_code == 204:
            # No new messages
            time.sleep(1)
        else:
            print(f"Failed to fetch messages: {response.status_code}, {response.text}")
            break
except KeyboardInterrupt:
    pass
finally:
    # Consumer 인스턴스 삭제
    response = requests.delete(
        consumer_instance["base_uri"],
        headers={"Accept": "application/vnd.kafka.v2+json"},
    )
    if response.status_code == 204:
        print("Consumer instance deleted successfully")
    else:
        print(
            f"Failed to delete consumer instance: {response.status_code}, {response.text}"
        )

Consumer instance created successfully
Subscribed to topic successfully
Received message: value1
Received message: value2
Received message: value3
Received message: value1
Received message: value2
Received message: value3
Received message: value4
Received message: value5
Received message: value6
Consumer instance deleted successfully
